# Pre-trained models & Transfer Learning

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
batch_size=2100
img_size = 299
train_path = '../data/sports/train/'
test_path = '../data/sports/test/'

In [ ]:
datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
bottleneck_generator = datagen.flow_from_directory(
    train_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    shuffle=False)

In [ ]:
images, labels  = bottleneck_generator.next()

In [ ]:
images.shape

In [ ]:
base_model = Xception(include_top=False,
                      weights='imagenet',
                      input_shape=(img_size, img_size, 3),
                      pooling='avg')

In [ ]:
bottlenecks = base_model.predict(images, verbose=2)

In [ ]:
bottlenecks.shape

In [ ]:
from sklearn.neighbors import DistanceMetric

In [ ]:
dist = DistanceMetric.get_metric('euclidean')

In [ ]:
def imshow_scaled(img):
    plt.imshow((img + 1) / 2)

In [ ]:
plt.subplot(1, 3, 1)
imshow_scaled(images[0])

plt.subplot(1, 3, 2)
imshow_scaled(images[1])

plt.subplot(1, 3, 3)
imshow_scaled(images[900])

plt.tight_layout()

In [ ]:
bn_dist = dist.pairwise(bottlenecks)

In [ ]:
bn_dist.shape

In [ ]:
plt.imshow(bn_dist, cmap='gray')

In [ ]:
imshow_scaled(images[0])

In [ ]:
dist_from_sel = pd.Series(bn_dist[0])

In [ ]:
dist_from_sel.sort_values().head(9)

In [ ]:
n_rows = 3
n_cols = 3
n_images = n_rows * n_cols

In [ ]:
retrieved = dist_from_sel.sort_values().head(n_images)

In [ ]:
plt.figure(figsize=(10, 10))
i = 1
for idx in retrieved.index:
    plt.subplot(n_rows, n_cols, i)
    imshow_scaled(images[idx])
    i += 1
plt.tight_layout()

## Exercise

Define a function `image_search` that encapsulates the code above and retrieves and plot the top 8 images closest to a given image index:

```python
def image_search(img_index, n_rows=3, n_columns=3):
    ....your code here...
```

In [ ]:
def image_search(img_index, n_rows=3, n_columns=3):
    n_images = n_rows * n_columns

    # create Pandas Series with distances from image
    dist_from_sel = pd.Series(bn_dist[img_index])
    
    # sort Series and get top n_images
    retrieved = dist_from_sel.sort_values().head(n_images)
    
    # create figure, loop over closest images indices 
    # and display them
    plt.figure(figsize=(10, 10))
    i = 1
    for idx in retrieved.index:
        plt.subplot(n_rows, n_cols, i)
        imshow_scaled(images[idx])
        if i == 1:
            plt.title('Selected image')
        else:
            plt.title("Dist: {:0.4f}".format(retrieved[idx]))
        i += 1
    plt.tight_layout()

In [ ]:
image_search(900)

In [ ]:
image_search(1600)

In [ ]:
image_search(100)